In [1]:
import xgt
import os
import pandas

from platform import python_version
print (python_version())

3.7.4


In [2]:
if os.environ.get('https_proxy'):
 del os.environ['https_proxy']
if os.environ.get('http_proxy'):
 del os.environ['http_proxy']

In [3]:
conn=xgt.Connection()
conn.server_version

'1.3.0'

In [4]:
try:
  devices = conn.get_vertex_frame('Devices')
except xgt.XgtNameError:
  devices = conn.create_vertex_frame(
      name='Devices',
      schema=[['device', xgt.TEXT]],
      key='device')
devices

In [5]:
try:
  netflow = conn.get_edge_frame('Netflow')
except xgt.XgtNameError:
  netflow = conn.create_edge_frame(
      name='Netflow',
      schema=[['epoch_time', xgt.INT],
              ['duration', xgt.INT],
              ['src_device', xgt.TEXT],
              ['dst_device', xgt.TEXT],
              ['protocol', xgt.INT],
              ['src_port', xgt.INT],
              ['dst_port', xgt.INT],
              ['src_packets', xgt.INT],
              ['dst_packets', xgt.INT],
              ['src_bytes', xgt.INT],
              ['dst_bytes', xgt.INT]],
      source=devices,
      target=devices,
      source_key='src_device',
      target_key='dst_device')
netflow

In [6]:
try:
  host_events = conn.get_edge_frame('HostEvents')
except xgt.XgtNameError:
  host_events = conn.create_edge_frame(
      name='HostEvents',
      schema=[['epoch_time', xgt.INT],
              ['event_id', xgt.INT],
              ['log_host', xgt.TEXT],
              ['user_name', xgt.TEXT],
              ['domain_name', xgt.TEXT],
              ['logon_id', xgt.INT],
              ['process_name', xgt.TEXT],
              ['process_id', xgt.INT],
              ['parent_process_name', xgt.TEXT],
              ['parent_process_id', xgt.INT]],
           source=devices,
           target=devices,
           source_key='log_host',
           target_key='log_host')
host_events

In [7]:
try:
  auth_events = conn.get_edge_frame('AuthEvents')
except xgt.XgtNameError:
  auth_events = conn.create_edge_frame(
           name='AuthEvents',
           schema = [['epoch_time',xgt.INT],
                     ['event_id',xgt.INT],
                     ['log_host',xgt.TEXT],
                     ['logon_type',xgt.INT],
                     ['logon_type_description',xgt.TEXT],
                     ['user_name',xgt.TEXT],
                     ['domain_name',xgt.TEXT],
                     ['logon_id',xgt.INT],
                     ['subject_user_name',xgt.TEXT],
                     ['subject_domain_name',xgt.TEXT],
                     ['subject_logon_id',xgt.TEXT],
                     ['status',xgt.TEXT],
                     ['src',xgt.TEXT],
                     ['service_name',xgt.TEXT],
                     ['destination',xgt.TEXT],
                     ['authentication_package',xgt.TEXT],
                     ['failure_reason',xgt.TEXT],
                     ['process_name',xgt.TEXT],
                     ['process_id',xgt.INT],
                     ['parent_process_name',xgt.TEXT],
                     ['parent_process_id',xgt.INT]],
            source = 'Devices',
            target = 'Devices',
            source_key = 'src',
            target_key = 'destination')
auth_events

In [8]:
# Utility to print the sizes of data currently in xGT
def print_data_summary():
  print('Devices (vertices): {:,}'.format(devices.num_vertices))
  print('Netflow (edges): {:,}'.format(netflow.num_edges))
  print('Host events (edges): {:,}'.format(host_events.num_edges))
  print('Authentication events (edges): {:,}'.format(auth_events.num_edges))
  print('Total (edges): {:,}'.format(
      netflow.num_edges + host_events.num_edges + auth_events.num_edges))
    
print_data_summary()

Devices (vertices): 0
Netflow (edges): 0
Host events (edges): 0
Authentication events (edges): 0
Total (edges): 0


In [9]:
%%time

# Load the HostEvents event data:
if host_events.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_1v.csv"]
    # urls = ["xgtd://nvme_data1/data_1v/wls_day-{:02d}_1v.csv".format(_) for _ in range(2,91)]
    urls = ["xgtd://nvme_data1/data_1v/wls_day-85_1v.csv"]
    # urls = ["xgtd://data_1v/wls_day-11_1v.csv"]
    host_events.load(urls)
    print_data_summary()

Devices (vertices): 13,491
Netflow (edges): 0
Host events (edges): 1,468,936,024
Authentication events (edges): 0
Total (edges): 1,468,936,024
CPU times: user 216 ms, sys: 308 ms, total: 524 ms
Wall time: 4min 19s


In [10]:
%%time

# Load the AuthEvents event data:
if auth_events.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_2v.csv"]
    # urls = ["xgtd://nvme_data9/data_2v/wls_day-{:02d}_2v.csv".format(_) for _ in range(2,91)]
    urls = ["xgtd://nvme_data1/data_2v/wls_day-85_2v.csv"]
    # urls = ["xgtd://data_2v/wls_day-11_2v.csv"]
    auth_events.load(urls)
    print_data_summary()

Devices (vertices): 18,925
Netflow (edges): 0
Host events (edges): 1,468,936,024
Authentication events (edges): 4,022,436,222
Total (edges): 5,491,372,246
CPU times: user 588 ms, sys: 374 ms, total: 962 ms
Wall time: 4min 52s


In [11]:
%%time

# Load the netflow data:
if netflow.num_edges == 0:
    #urls = ["https://datasets.trovares.com/LANL/xgt/nf_day-85.csv"]
    #urls = ["xgtd://nvme_data1/data_nf/nf_day-{:02d}.csv".format(_) for _ in range(2,91)]
    urls = ["xgtd://nvme_data1/data_nf/nf_day-85.csv"]
    # urls = ["xgtd://nvme_data1/data_nf/nf_day-11.csv"]
    netflow.load(urls)
    print_data_summary()

Devices (vertices): 933,714
Netflow (edges): 17,882,795,024
Host events (edges): 1,468,936,024
Authentication events (edges): 4,022,436,222
Total (edges): 23,374,167,270
CPU times: user 1.55 s, sys: 990 ms, total: 2.54 s
Wall time: 14min 47s


In [12]:
# Utility function to launch queries and show job number:
#   The job number may be useful if a long-running job needs
#   to be canceled.

def run_query(query, table_name = "answers", drop_answer_table=True, show_query=False):
    if drop_answer_table:
        conn.drop_frame(table_name)
    if query[-1] != '\n':
        query += '\n'
    query += 'INTO {}'.format(table_name)
    if show_query:
        print("Query:\n" + query)
    job = conn.schedule_job(query)
    print("Launched job {}".format(job.id))
    conn.wait_for_job(job)
    table = conn.get_table_frame(table_name)
    return table

In [13]:
# Generate a new edge frame for holding only the RDP edges
import time
query_start_time = time.time()

conn.drop_frame('RDPFlow')
rdp_flow = conn.create_edge_frame(
            name='RDPFlow',
            schema=netflow.schema,
            source=devices,
            target=devices,
            source_key='src_device',
            target_key='dst_device')
rdp_flow

In [14]:
%%time

#Filter out all the Nodes in which 4768(TGT request has been logged)

q = """
MATCH (n1:Devices)-[r:AuthEvents]->[n2:Devices]
WHERE r.event_id = 4768
CREATE (n1)-[r1:tgt_req_events {epoch_time:r.epoch_time, event_id:r.event_id,
 log_host:r.log_host, logon_type:r.logon_type,
 logon_type_description:r.logon_type_description, user_name:r.user_name,
 domain_name:r.domain_name, logon_id:r.logon_id,
 subject_user_name:r.subject_user_name, subject_domain_name:r.subject_domain_name,
 subject_logon_id:r.subject_logon_id, status:r.status,
 src:r.src, service_name:r.service_name,
 destination:r.destination, authentication_package:r.authentication_package,
 failure_reason: r.failure_reason, process_name:r.process_name,
 process_id:r.process_id, parent_process_name:r.parent_process_name,
 parent_process_id:r.parent_process_id}]->(n2)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

Launched job 3914
Number of answers: 2,176
CPU times: user 304 ms, sys: 165 ms, total: 469 ms
Wall time: 1min 53s


In [15]:
%%time

#Filter out all the Nodes in which 4769(Service ticket request has been logged)
q = """
MATCH (n1:Devices)-[r:AuthEvents]->[n2:Devices]
WHERE r.event_id = 4769
CREATE (n1)-[r1:service_ticket_req {epoch_time:r.epoch_time, event_id:r.event_id,
 log_host:r.log_host, logon_type:r.logon_type,
 logon_type_description:r.logon_type_description, user_name:r.user_name,
 domain_name:r.domain_name, logon_id:r.logon_id,
 subject_user_name:r.subject_user_name, subject_domain_name:r.subject_domain_name,
 subject_logon_id:r.subject_logon_id, status:r.status,
 src:r.src, service_name:r.service_name,
 destination:r.destination, authentication_package:r.authentication_package,
 failure_reason: r.failure_reason, process_name:r.process_name,
 process_id:r.process_id, parent_process_name:r.parent_process_name,
 parent_process_id:r.parent_process_id}]->(n2)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

Launched job 5036
Number of answers: 755,260
CPU times: user 58.3 ms, sys: 42 ms, total: 100 ms
Wall time: 53 s
